In [2]:
!pip install stanza
!pip install dateparser
!pip install spacy
!python -m spacy download en_core_web_trf
!pip install earthengine-api
!pip install geemap
!pip install pandas nltk
!pip install gtts
!pip install stt
!pip install earthengine-api
!pip install ipygee
!pip install ffmpeg-python
!pip install wheel
!pip install pipwin
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install PyAudio
!pip install pyttsx3
!pip install deepgram-sdk


2023-11-28 10:11:10.766499: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 10:11:10.766560: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 10:11:10.766606: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 10:11:10.787216: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-28 10:11:13.115471: W tensorflow/compiler/

In [3]:
!pip install SpeechRecognition

In [4]:

import deepgram
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
dg_client = deepgram.Deepgram('e4712c88fedd671f10513629ac6c98015e602d5d')
#response = await Deepgram.keys.create(PROJECT_ID, COMMENT_FOR_KEY, SCOPES)
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr


from scipy.io.wavfile import write as wav_write




In [5]:
import ee
import geemap
import stanza
import dateparser
from datetime import datetime
import spacy
import ee
import geemap
from datetime import datetime, timedelta
import nltk
nltk.download('punkt')
import pandas as pd
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
ee.Authenticate()
ee.Initialize()


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=toK73DJZpI_ZvS3541Pev0B-mywkJXEiJNpsBf7yVes&tc=aAbKG2flk7Sa9ahk3Bzd1r9oGYaS1RHeMAgmDNoW-CY&cc=DFLWnz2Yuyv5vftf8ziVQAhlH-bXSQGKHzlclh-l7DE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXl3YQWB0ckLFKvHtiYtkwBdyDoW5eoEBof9npJVEYBvUoxejPZHiYw

Successfully saved authorization token.


In [7]:
from difflib import get_close_matches

def date_parser(text):
    # Download and set up the neural pipeline for English
    stanza.download('en')
    nlp = stanza.Pipeline('en')

    # Process the input text using Stanza NLP
    doc = nlp(text)

    # Extract DATE entities
    ents = []
    for sent in doc.sentences:
        for ent in sent.ents:
            if ent.type == 'DATE':
                ents.append(ent.text)

    # Parse the dates using dateparser
    parsed_dates = [dateparser.parse(date) for date in ents]

    # Sort the parsed dates in ascending order
    sorted_dates = sorted(parsed_dates)

    # Set the smallest date as the start_date and the largest date as the end_date
    start_date = sorted_dates[0].strftime('%Y-%m-%d') if sorted_dates else None
    end_date = sorted_dates[-1].strftime('%Y-%m-%d') if sorted_dates else None

    # If both start_date and end_date are the same, set end_date to today's date
    if start_date == end_date and start_date is not None:
        end_date = datetime.now().strftime('%Y-%m-%d')

    # If both start_date and end_date are None, set end_date to today's date and start_date to past 14 days
    if start_date is None and end_date is None:
        end_date = datetime.now().strftime('%Y-%m-%d')
        start_date = (datetime.now() - timedelta(days=14)).strftime('%Y-%m-%d')

    return start_date, end_date

# Module 2: ROI Extract Code

import spacy
import pandas as pd
from difflib import get_close_matches
from nltk.tokenize import word_tokenize

class LocationExtractor:

    def __init__(self, csv_file_path, threshold=0.8, default_entity="kadam"):
        # Load the English language model for spaCy
        self.nlp = spacy.load('en_core_web_trf')
        # Load the CSV file for fuzzy matching
        self.df = pd.read_csv(csv_file_path)
        self.threshold = threshold
        self.default_entity = "alisagar"

    def fuzzy_match(self, token):
        # Use difflib to find the closest match
        matches = get_close_matches(token, self.df['ROI_Name'].str.lower().tolist(), n=1, cutoff=self.threshold)

        # Check if there is a match
        if matches:
            return matches[0]
        else:
            return None

    def extract_entities(self, user_input):
        # Convert user input to lowercase
        user_input_lower = user_input.lower()
        # Tokenize the user input
        tokens = word_tokenize(user_input_lower)

        # Initialize variables to store the best matching entity
        best_entity = None

        # Iterate through the tokens and perform fuzzy matching
        for token in tokens:
            matched_entity = self.fuzzy_match(token)
            if matched_entity:
                best_entity = matched_entity
                break  # Stop after finding the first match

        # Return the best matching entity or the default if no match is found
        return best_entity if best_entity else self.default_entity






In [23]:
import ee
import geemap
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import pandas as pd  # Don't forget to import pandas
current_date_time = datetime.now()

# Define the SARAnalyzer class
class MapVisualizer:
    def __init__(self):
        self.sar_collection = None
        self.selected_roi = None
        self.start_date = None
        self.end_date = None

    def import_and_add_layers(self, asset_id, predefined_layers=None):
        shp = ee.FeatureCollection(asset_id)

        if predefined_layers:
            shp = shp.map(lambda feature: feature.set(predefined_layers))

        return shp

    def add_sar_layer_to_roi(self, shapefile, start_date, end_date, map_obj):
        sar_collection = self.load_sar_collection(start_date, end_date)

        sar_vv = sar_collection.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
            .filter(ee.Filter.eq('instrumentMode', 'IW')).mean().clip(shapefile.geometry())

        map_obj.addLayer(sar_vv, {'bands': ['VV'], 'min': -20, 'max': 0, 'gamma': 1.4}, 'Clipped SAR (VV) Layer')

        return sar_vv

    def load_sar_collection(self, start_date, end_date):
        sar_collection = ee.ImageCollection('COPERNICUS/S1_GRD')\
            .filterDate(ee.Date(start_date), ee.Date(end_date))\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
            .filter(ee.Filter.eq('instrumentMode', 'IW'))

        self.sar_collection = sar_collection
        return sar_collection

    def calculate_water_spread(self, image, threshold):
        water_mask = image.lt(threshold)

        water_area_m2 = water_mask.multiply(ee.Image.pixelArea()).reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=self.selected_roi.geometry(),
            scale=30
        ).getInfo()['VV']

        water_area_km2 = water_area_m2 / 1e6

        return water_area_km2

    def calculate_yearly_water_spread(self, image_collection, threshold):
        yearly_water_spread = []

        for year in range(self.start_date.year, self.end_date.year + 1):
            start_date = f"{year}-01-01"
            end_date = f"{year + 1}-01-01"

            year_collection = image_collection.filterDate(ee.Date(start_date), ee.Date(end_date))
            yearly_water_spread.append(self.calculate_water_spread(year_collection.mean(), threshold))

        return yearly_water_spread

    def calculate_max_water_spread(self, selected_roi_name):
        # Set a large date range to cover all available data
        start_date_max = "2015-01-01"
        end_date_max = "2023-10-01"
        sar_collection_max = self.load_sar_collection(start_date_max, end_date_max)

        # Calculate maximum water spread in the ROI
        sar_vv_max = self.add_sar_layer_to_roi(self.selected_roi, start_date_max, end_date_max, geemap.Map())
        max_water_spread = self.calculate_water_spread(sar_vv_max, -15)

        return max_water_spread

    def compare_water_spread(self, water_spread_user_input, max_water_spread):
        conclusion = ""
        if water_spread_user_input > max_water_spread:
            conclusion += f'Water spread increased in the user input duration by {water_spread_user_input - max_water_spread:.2f} square kilometers.\n'
        elif water_spread_user_input < max_water_spread:
            conclusion += f'Water spread decreased in the user input duration by {max_water_spread - water_spread_user_input:.2f} square kilometers.\n'
        else:
            conclusion += 'Water spread remained the same in the user input duration.\n'

        return conclusion

    def run_analysis(self, asset_ids, selected_roi_name, start_date, end_date, csv_file_path):
        conclusion = ""
        df = pd.read_csv("/content/SF.csv")
        valid_roi_names = df['ROI_Name'].tolist()
        if selected_roi_name in valid_roi_names:
            selected_roi_index = valid_roi_names.index(selected_roi_name)
            self.selected_roi = self.import_and_add_layers(asset_ids[selected_roi_index])

            static_map = geemap.Map(width=800, height=600)
            sar_vv = self.add_sar_layer_to_roi(self.selected_roi, start_date, end_date, static_map)
            static_map.centerObject(self.selected_roi, 10)
            display(static_map)

            # Set the start_date and end_date attributes
            self.start_date = datetime.strptime(start_date, "%Y-%m-%d")
            self.end_date = datetime.strptime(end_date, "%Y-%m-%d")

            # Calculate water spread for the user input duration
            water_spread_user_input = round(self.calculate_water_spread(sar_vv, -15))
            conclusion += f'Water spread for user input duration ({selected_roi_name.capitalize()}): {water_spread_user_input:} square kilometers\n'

            # Calculate maximum water spread in the ROI
            max_water_spread =round( self.calculate_max_water_spread(selected_roi_name))
            conclusion += f'Maximum water spread ({selected_roi_name.capitalize()}): {max_water_spread:} square kilometers\n'

            # Compare water spread for user input duration with maximum water spread
            conclusion += self.compare_water_spread(water_spread_user_input, max_water_spread)



        else:
            conclusion += "Invalid ROI name. Please enter a valid ROI name.\n"

        return static_map, conclusion

In [24]:
import speech_recognition as sr

csv_file_path = '/content/SF.csv'
df = pd.read_csv(csv_file_path)
asset_ids = df['ROI_path'].tolist()
csv_file_path = '/content/SF.csv'


audio, sr = get_audio()
# Save the audio to a WAV file
wav_write("audio.wav", sr, audio)

import speech_recognition as sr

# Initialize the recognizer
recognizer = sr.Recognizer()

# Capture audio from the microphone
with sr.AudioFile("audio.wav") as source:
    print("Listening... Speak now!")
    audio = recognizer.listen(source)
try:
    print("Transcribing...")
    user_text = recognizer.recognize_google(audio, language="en-IN")
    print("Transcription: ", user_text)
except sr.UnknownValueError:
    print("Sorry, could not understand the audio.")
except sr.RequestError as e:
    print("Could not request results; {0}".format(e))

start_date, end_date = date_parser(user_text)
print("Start Date:", start_date)
print("End Date:", end_date)

    # Module 2: ROI Extract Code
location_extractor = LocationExtractor(csv_file_path)

selected_roi_name = location_extractor.extract_entities(user_text)
print("ROI Name:", selected_roi_name)

    # Module 3: Map Visualizer Code
map_visualizer = MapVisualizer()
static_map, conclusion = map_visualizer.run_analysis(asset_ids, selected_roi_name, start_date, end_date, csv_file_path)

static_map
print(conclusion)




Listening... Speak now!
Transcribing...
Transcription:  is the water spread level of kadam


INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| pos          | combined_charlm     |
| lemma        | combined_nocharlm   |
| constituency | ptb3-revised_charlm |
| depparse     | combined_charlm     |
| sentiment    | sstplus             |
| ner          | ontonotes_charlm    |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


Start Date: 2023-11-14
End Date: 2023-11-28
ROI Name: kadam


Map(center=[19.130636472341664, 78.74430847265431], controls=(WidgetControl(options=['position', 'transparent_…

Water spread for user input duration (Kadam): 12.00 square kilometers
Maximum water spread (Kadam): 19.00 square kilometers
Water spread decreased in the user input duration by 7.00 square kilometers.



In [25]:
from gtts import gTTS #Import Google Text to Speech
from IPython.display import Audio
#Import Audio method from IPython's Display Class
tts = gTTS(conclusion) #Provide the string to convert to speech
tts.save('conclusion.wav') #save the string converted to speech as a .wav file
sound_file = 'conclusion.wav'
Audio(sound_file, autoplay=True)